# Generalized T-Maze environment

In [ ]:
%load_ext autoreload
%autoreload 2

from pymdp.jax.envs.generalized_tmaze import (
    GeneralizedTMazeEnv, parse_maze, render 
)
from pymdp.jax.envs.rollout import rollout
from pymdp.jax.agent import Agent

import numpy as np 
import jax.random as jr 
import jax.numpy as jnp
import jax.tree_util as jtu 

### Create the environment

In this example we create a simple square environment, where multiple cues are present, and multiple reward pairs. Each cue indicates the location of one of the reward pairs. 

The agent is can move in the grid world using actions up, down, left and right, and observes the current tile it is at. 

The grid world is specified by a matrix using the following labels: 

```
0: Empty space
1: The initial position of the agent
2: Walls
3 + i: Cue for reward i
4 + i: Potential reward location i 1
4 + i: Potential reward location i 2
```

In [ ]:
def get_maze_matrix(small=False):
    if small:
        M = np.zeros((3, 5))

        # Set the reward locations
        M[0,1] = 4
        M[1,1] = 5
        M[1,3] = 7
        M[0,3] = 8

        # Set the cue locations
        M[2,0] = 3
        M[2,4] = 6

        # Set the initial position
        M[2,3] = 1
    else:

        M = np.zeros((5, 5))

        # Set the reward locations
        M[0,1] = 4
        M[1,1] = 5
        M[1,3] = 7
        M[0,3] = 8
        M[4,1] = 10
        M[4,3] = 11

        # Set the cue locations
        M[2,0] = 3
        M[2,4] = 6
        M[3,2] = 9

        # Set the initial position
        M[2,2] = 1
    return M

M = get_maze_matrix(small=False)
env_info = parse_maze(M)
tmaze_env = GeneralizedTMazeEnv(env_info)
_ = render(env_info, tmaze_env)

#### Create the agent. 

The PyMDPEnv class consists of a params dict that contains the A, B, and D vectors of the environment. We initialize our agent using the same parameters. This means that the agent has full knowledge about the environment transitions, and likelihoods. We initialize the agent with a flat prior, i.e. it does not know where it, or the reward is. Finally, we set the C vector to have a preference only over the rewarding observation of cue-reward pair 1 (i.e. C[1][1] = 1 and zero for other values). 

In [ ]:
A = [a.copy() for a in tmaze_env.params["A"]]
B = [b.copy() for b in tmaze_env.params["B"]]
A_dependencies = tmaze_env.dependencies["A"]
B_dependencies = tmaze_env.dependencies["B"]

# [position], [cue], [reward]
C = [jnp.zeros(a.shape[:2]) for a in A]

rewarding_modality = 2 + env_info["num_cues"]
#rewarding_modality = -1

C[rewarding_modality] = C[rewarding_modality].at[:,1].set(2.0)
C[rewarding_modality] = C[rewarding_modality].at[:,2].set(-3.0)


D = [jnp.ones(b.shape[:2]) for b in B]

agent = Agent(
    A, B, C, D, 
    None, None, None, 
    policy_len=7,
    A_dependencies=A_dependencies, 
    B_dependencies=B_dependencies,
    apply_batch=False
)



### Rollout an agent episode 

Using the rollout function, we can run an active inference agent in this environment over a specified number of discrete timesteps using the parameters previously set. 

In [ ]:
key = jr.PRNGKey(0)
_, info, _ = rollout(agent, tmaze_env, num_timesteps=15, rng_key=key)

In [ ]:
ims = []
for t in range(15): 
    print(f'Time t={t}')
    env_state = jtu.tree_map(lambda x: x[:, t], info['env'])
    ims.append(render(env_info, env_state))
ims = [np.array(i) for i in ims]

In [ ]:
import imageio as io

io.mimsave('tmp_dir/gif.gif', ims, format="GIF", duration=1)